# Crear un TorchPlayer


Recibe el modelo a instanciar como path y juega con el mismo

- Pensar como resolver el problema de que solo samplee las válidas
- Agregarle la opción de monte carlo tree search (opcional) con las opciones de iterationLimit, timeLimit

Si va a agregar MCTS mirar la notebook 007_MCTS.ipnb

In [9]:
from players import BasePlayer
from boardgame2 import ReversiEnv
import numpy as np
from typing import Union
from stable_baselines3 import PPO

In [10]:
class TorchPlayer(BasePlayer):
    def __init__(self,
                 model_path: str,
                 player: int = 1,
                 board_shape: int = None,
                 env: ReversiEnv = None,
                 flatten_action: bool = False,
                 deterministic: bool=True,
                 only_valid: bool=True,
                 mcts: bool=False,
                 iterationLimit: int=None,
                 timeLimit: int=None
                 ):
        super.__init__(player, board_shape, env, flatten_action)
        self.model = PPO.load(model_path)
        self.deterministic = deterministic
        self.only_valid = only_valid
        self.mcts = mcts
        self.iterationLimit = iterationLimit
        self.timeLimit = timeLimit

    def predict(self, board: np.ndarray) -> Union[int, np.ndarray]:
        obs = board
        if self.only_valid:
            obs = [board, self.env.get_valid((board, 1))]
        # The model expects a batch of observations.
        # Make a batch of 1 obs
        obs = [obs]
        action = self.model.predict(obs, deterministic=self.deterministic)[0]
        if self.flatten_action:
            return action
        else:
            return np.array([action // self.board_shape, action % self.board_shape])

# Arena

Testear el jugador contra los distintos jugadores